In [2]:
import pandas as pd 
import json
from pycel import ExcelCompiler

def get_graphs():        
    # Преобразование JSON в словарь
    with open("./JSON/graph.json", "r", encoding="utf8") as file:
        data = json.load(file)

    # Имя файла Excel
    file_name = "Excels\Производство глинозема"

    # Создание списка DataFrame'ов
    dfs = []
    # Обработка каждой категории в JSON-объекте
    for category, inner_dict in data.items():
        df = pd.DataFrame()
        for i in ["1", "2"]:
            # Обработка каждого значения внутри текущей категории
            for value_name, cell_coordinates in inner_dict.items():
                # Получение значения из ячейки
                cell_value = get_cell_value(file_name + i + ".xlsx", cell_coordinates)
                
                # Добавление значения в DataFrame
                df.loc[category + i ,value_name] = cell_value
            
            # Добавление DataFrame в список
        dfs.append(df)
    return dfs

def get_cell_value(file_name, cell_coordinates):
    # Создание компилятора Excel
    compiler = ExcelCompiler(filename=file_name)
    
    # Разделение координат на столбец и строку
    column_letter = cell_coordinates[0]
    row = int(cell_coordinates[1:])
    
    # Определение листа и значения ячейки
    sheet_name = "Таблицы"
    cell_value = compiler.evaluate(f'{sheet_name}!{column_letter}{row}')
    
    return cell_value

dfs = get_graphs()

# Список DataFrame
list_of_dfs = [dfs[0], dfs[1]]

# Соединяем все DataFrame в один
df = pd.concat(list_of_dfs)

In [3]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

def plot_dataframe(df):
    stages = ['Спекание', 'Выщелачивание', 'Обескремнивание', 'Карбонизация', 'Кальцинация']
    types = ['1', '2']

    fig = go.Figure()

    # ширина баров на графике
    width = 0.35

    for i, type in enumerate(types):
        for j, stage in enumerate(stages):
            al_key = f'Содержание Al2O3, кг/т глинозема{type}'
            mass_key = f'Масса материального потока, кг/т глинозема {type}'
            
            # x-координаты для группы столбцов
            x = np.arange(len(stages)) + i * width - (len(types) - 1) * width / 2
            
            fig.add_trace(go.Bar(
                x=[x[j]],
                y=[df.loc[mass_key, stage]],
                width=width,
                name=f'Масса, глинозем {type}' if j == 0 else None,  # Добавляем имя только к первому трейсу
                marker_color=['lightsalmon', 'lightblue'][i],
                text=round(df.loc[mass_key, stage]),  # Округляем значения
                textposition='auto',
                showlegend=j == 0  # Показываем легенду только для первого трейса
            ))

            fig.add_trace(go.Bar(
                x=[x[j]],
                y=[df.loc[al_key, stage]],
                width=width,
                name=None,  # Скрываем имя
                marker_color=['indianred', 'darkblue'][i],
                text=round(df.loc[al_key, stage]),  # Округляем значения
                textposition='auto',
                showlegend=False  # Скрываем легенду
            ))

    fig.update_layout(
        title='Сравнение содержания Al2O3 и массы потока на каждом этапе',
        xaxis=dict(
            tickmode='array',
            tickvals=np.arange(len(stages)),
            ticktext=stages,
            title='Этап',
            # titlefont_size=20,
            tickfont_size=19,
            tickangle=0,
        ),
        yaxis=dict(
            title='Масса потока',
            # titlefont_size=20,
            tickfont_size=20,
        ),
        legend=dict(
            x=0,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        barmode='overlay',
        bargap=0.15,
        bargroupgap=0.1,
        autosize=False,
        width=1000,  # Увеличиваем ширину графика
        height=700,  # Увеличиваем высоту графика
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=50,
            pad=0
        )
    )
    
    fig.update_layout(
    font=dict(
        size=16,  # Set the font size here
    )
    )
    return fig

# Затем используйте эту функцию, как указано в предыдущем сообщении.

plot_dataframe(df)

In [4]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QTableWidget, QTableWidgetItem, QHeaderView, QWidget
from PyQt5.QtCore import Qt

class MyTable(QTableWidget):
    def __init__(self, r = 5, c = 3):
        super().__init__(r, c)
        self.init_ui()

    def init_ui(self):
        self.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        self.verticalHeader().setSectionResizeMode(QHeaderView.Stretch)

        self.horizontalHeader().setVisible(False) # hiding column names
        self.verticalHeader().setVisible(False) # hiding row names

        self.setSpan(0, 0, 2, 1)
        self.setSpan(0, 1, 1, 2)

        self.setItem(0, 0, QTableWidgetItem("Показатель"))
        self.setItem(0, 1, QTableWidgetItem("Показатель"))

        self.setItem(2, 0, QTableWidgetItem("Марка глинозема"))
        self.setItem(3, 0, QTableWidgetItem("Товарный выход, %"))
        self.setItem(4, 0, QTableWidgetItem("Кол-во отвального шлама, кг/т тов. гл."))

        self.setItem(1, 1, QTableWidgetItem("Боксит 1"))
        self.setItem(1, 2, QTableWidgetItem("Боксит 2"))

        self.setItem(2, 1, QTableWidgetItem("Г-000"))
        self.setItem(2, 2, QTableWidgetItem("Г-000"))

        self.setItem(3, 1, QTableWidgetItem("87,0"))
        self.setItem(3, 2, QTableWidgetItem("87,0"))

        self.setItem(4, 1, QTableWidgetItem("1700"))
        self.setItem(4, 2, QTableWidgetItem("1600"))


class MainWindow(QMainWindow):
    def __init__(self, *args, **kwargs):
        super(MainWindow, self).__init__(*args, **kwargs)
        
        self.table_widget = MyTable()
        
        self.setCentralWidget(self.table_widget)


app = QApplication(sys.argv)
main = MainWindow()
main.show()

sys.exit(app.exec_())


SystemExit: 0

In [ ]:
def boxite2():
    dasd